# Wikipedia and more

There's a whole wiki-verse that is just confusing
https://doc.wikimedia.org

MediaWiki is for edits, which I don't want to do. 
I believe you don't read data with API key. 

In [25]:
from rag_eng import read_dot_env
import requests
import os
import json
from typing import List, Optional
import base64
from pathlib import Path
import pandas as pd

In [26]:
read_dot_env()
access_token = os.getenv("WIKI_MEDIA_ACCESS_TOKEN")
assert access_token is not None
user_agent = os.getenv("WIKI_MEDIA_USER_AGENT")
assert user_agent is not None
client_id = os.getenv("WIKI_MEDIA_CLIENT_ID")
assert client_id is not None
client_secret = os.getenv("WIKI_MEDIA_CLIENT_SECRET")
assert client_secret is not None

In [27]:
def url_builder(protocol: str, subdomains: Optional[List[str]], second_level_domain: str, top_level_domain: str, sub_directories: Optional[List[str]]):
    assert protocol == 'http' or protocol == 'https'
    assert second_level_domain is not None and type(second_level_domain) is str
    assert top_level_domain is not None and type(top_level_domain) is str
    url = f"{protocol}://"
    if subdomains is not None and len(subdomains) > 0:
        for sub in subdomains:
            assert type(sub) is str
            url = f"{url}{sub}."
    url = f"{url}{second_level_domain}.{top_level_domain}"
    if sub_directories is not None and len(sub_directories) > 0:
        for sub in sub_directories:
            assert type(sub) is str
            url = f"{url}/{sub}"
    return url


## Failing token

In [ ]:
# Request Authorization

# auth_headers = {
#     'Content-Type': 'application/json',
#     'User-Agent': user_agent
# }

# auth_params = {
#     'client_id': client_id,
#     'response_type': 'code'
# }

# # Get token
# auth_response = requests.get(
#     'https://meta.wikimedia.org/w/rest.php/oauth2/authorize',
#     headers=auth_headers,
#     params=auth_params)

# if auth_response.status_code == 200:
#     jres = auth_response.json()
#     print(jres)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# print(auth_response.text)

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-disabled vector-feature-custom-font-size-clientpref--excluded vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-disabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-not-available" lang="en" dir="ltr">
<head>
<meta charset="UTF-8">
<title>Log in - Meta</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-disabled vector-fe

In [ ]:
# Get access token using client credentials
# Done above

## I don't think you use API access tokens. seems you just can't
## https://doc.wikimedia.org

# Create basic auth header
# credentials = base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()

# token_headers = {
#     'Authorization': f'Basic {credentials}',
#     'Content-Type': 'application/x-www-form-urlencoded',
#     'User-Agent': 'YourAppName/1.0 (your@email.com)'
# }

# token_data = {
#     'grant_type': 'client_credentials'
# }

# # Get token
# token_response = requests.post(
#     'https://meta.wikimedia.org/w/rest.php/oauth2/access_token',
#     headers=token_headers,
#     data=token_data)

# if token_response.status_code == 200:
#     access_token = token_response.json()['access_token']
    
#     # Now use this token for API calls
#     api_headers = {
#         'Authorization': f'Bearer {access_token}',
#         'User-Agent': 'YourAppName/1.0 (your@email.com)'
#     }
    
#     response = requests.get('https://api.wikimedia.org/core/v1/wikipedia/en/search/page', 
#                            headers=api_headers, 
#                            params={'q': 'python'})
#     print(response.json())
# else:
#     print("Token request failed:", token_response.text)

{'errorKey': 'mwoauth-invalid-authorization-invalid-user', 'messageTranslations': {'en': 'The authorization headers in your request are for a user that does not exist here'}, 'httpCode': 403, 'httpReason': 'Forbidden'}


The below was failing with my access token so I created the above but you do get 500 free requests per hour so default to the free API for now.

[wikimedia docs](https://api.wikimedia.org/wiki/Searching_for_Wikipedia_articles_using_Python) are good to help.

In [28]:
language_code = 'en'
search_query = 'donald trump epstein'
number_of_results = 1
headers = {
    # "Authorization": f"Bearer {access_token.strip()}",
    "User-Agent": user_agent,
    "Accept": "*/*",
}

url = url_builder('https', ['api'], 'wikimedia', 'org', ['core', 'v1', 'wikipedia', 'en', 'search', 'page'])
params = {'q': search_query, 'limit': number_of_results}

print(url)

response = requests.get(url, headers=headers, params=params)

https://api.wikimedia.org/core/v1/wikipedia/en/search/page


In [32]:
response
print(dir(response))
print(response.status_code)

['__attrs__', '__bool__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__nonzero__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_content', '_content_consumed', '_next', 'apparent_encoding', 'close', 'connection', 'content', 'cookies', 'elapsed', 'encoding', 'headers', 'history', 'is_permanent_redirect', 'is_redirect', 'iter_content', 'iter_lines', 'json', 'links', 'next', 'ok', 'raise_for_status', 'raw', 'reason', 'request', 'status_code', 'text', 'url']
200


In [33]:
pages = response.json().get('pages')
assert pages is not None

In [35]:
# Get First Page
page_path = pages[0].get('key')
assert page_path is not None

https://stackoverflow.com/questions/7185288/how-can-i-get-wikipedia-content-using-wikipedias-api

https://www.mediawiki.org/wiki/API:Properties#revisions_.2F_rv

https://www.mediawiki.org/wiki/API:Get_the_contents_of_a_page#Method_3:_Use_the_TextExtracts_API

Then found this helpful person: https://github.com/mudroljub/wikipedia-api-docs

https://www.mediawiki.org/wiki/Special:ApiSandbox

In [36]:
language_code = 'en'
headers = {
    # "Authorization": f"Bearer {access_token.strip()}",
    "User-Agent": user_agent,
    "Accept": "*/*",
}

url = url_builder('https', [language_code], 'wikipedia', 'org', ['w', 'api.php'])
params = {
    'action': 'parse', 
    "format": "json",
    "page": page_path,
    "prop": "text", # Gets the HTML in { "parse": {"title": ..., "text": { "*": <html> }}}
}

print(url)

response = requests.get(url, headers=headers, params=params)



https://en.wikipedia.org/w/api.php


In [38]:
response.json()

{'parse': {'title': 'Relationship of Donald Trump and Jeffrey Epstein',
  'pageid': 80511968,
  'text': {'*': '<div class="mw-content-ltr mw-parser-output" lang="en" dir="ltr"><p class="mw-empty-elt">\n\n</p>\n<figure class="mw-default-size mw-halign-right" typeof="mw:File/Thumb"><span><video id="mwe_player_0" poster="//upload.wikimedia.org/wikipedia/commons/thumb/8/86/President_Trump_Participates_in_a_Bilateral_Meeting_with_the_Amir_of_the_State_of_Qatar.webm/250px-seek%3D237-President_Trump_Participates_in_a_Bilateral_Meeting_with_the_Amir_of_the_State_of_Qatar.webm.jpg" controls="" preload="none" data-mw-tmh="" class="mw-file-element" width="250" height="141" data-durationhint="292" data-mwtitle="President_Trump_Participates_in_a_Bilateral_Meeting_with_the_Amir_of_the_State_of_Qatar.webm" data-mwprovider="wikimediacommons" resource="/wiki/File:President_Trump_Participates_in_a_Bilateral_Meeting_with_the_Amir_of_the_State_of_Qatar.webm"><source src="//upload.wikimedia.org/wikipedia/c

In [6]:
data_path = Path('../') / 'judge_model' / ".datasets" / "processed" / "fever_train_chunk_0000.parquet"
print(data_path.exists())
dft = pd.read_parquet(data_path)
dft.head(5)

True


,id,claim,evidence,label,challenge
0,14802,Asiatic Society of Bangladesh(housed in Nimtal...,"[The society is housed in Nimtali, walking dis...",SUPPORTS,other
1,28540,"Lindfield railway station has 3 bus routes, in...",[Lindfield Station is served by three bus rout...,SUPPORTS,other
2,71874,Mukaradeeb('Wolf's Den') is a city in Iraq nea...,['Wolf's Den') is a small village in Iraq near...,SUPPORTS,combining tables and text
3,70296,Herbivore men was coined by Maki Fukasawa and ...,[The term was coined by the author Maki Fukasa...,SUPPORTS,multi-hop reasoning
4,16578,"Shulin, a 33.1288 km (12.7911 sq mi) land loca...",['forest district') is an inner city district ...,REFUTES,other


In [7]:
dft['label'].value_counts()

label
SUPPORTS           31811
REFUTES            14610
NOT ENOUGH INFO     1784
Name: count, dtype: int64

In [21]:
jsondata_path = Path('../') / 'judge_model' / ".datasets" / "original" / "feverous_train_challenges.jsonl"
with open(jsondata_path, 'r') as file:
    ln = 1
    cnt = 0
    for line in file.readlines():
        datum = json.loads(line)
        # print(json.dumps(datum, indent=2))
        ev = datum.get('evidence')
        if ev is None or type(ev) is not list:
            ev = []
        # print(len(ev))
        if len(ev) > 2:
            print(f"{ln}) {len(ev)}")
            print(json.dumps(ev))
            print([x.get('content') for x in ev])
            cnt += 1
        ln += 1
        if cnt > 10:
            break

279) 3
[{"content": ["Dale Rudge_cell_0_4_1", "Dale Rudge_cell_0_2_1"], "context": {"Dale Rudge_cell_0_4_1": ["Dale Rudge_title", "Dale Rudge_header_cell_0_4_0", "Dale Rudge_header_cell_0_0_0"], "Dale Rudge_cell_0_2_1": ["Dale Rudge_title", "Dale Rudge_header_cell_0_2_0", "Dale Rudge_header_cell_0_0_0"]}}, {"content": ["Dale Rudge_sentence_1", "Dale Rudge_sentence_0"], "context": {"Dale Rudge_sentence_1": ["Dale Rudge_title"], "Dale Rudge_sentence_0": ["Dale Rudge_title"]}}, {"content": ["Dale Rudge_sentence_6", "Dale Rudge_cell_0_2_1"], "context": {"Dale Rudge_sentence_6": ["Dale Rudge_title"], "Dale Rudge_cell_0_2_1": ["Dale Rudge_title", "Dale Rudge_header_cell_0_2_0", "Dale Rudge_header_cell_0_0_0"]}}]
[['Dale Rudge_cell_0_4_1', 'Dale Rudge_cell_0_2_1'], ['Dale Rudge_sentence_1', 'Dale Rudge_sentence_0'], ['Dale Rudge_sentence_6', 'Dale Rudge_cell_0_2_1']]
337) 3
[{"content": ["Maguarichi Municipality_sentence_0"], "context": {"Maguarichi Municipality_sentence_0": ["Maguarichi Muni

In [22]:
data_path = Path('../') / 'judge_model' / ".datasets" / "updated" / "fever_train_titles_chunk_0000.parquet"
print(data_path.exists())
dft = pd.read_parquet(data_path)
dft.head(5)

True


,id,titles,pages
0,14802,[Asiatic Society of Bangladesh],[Asiatic_Society_of_Bangladesh]
1,28540,[Lindfield railway station],[Lindfield_railway_station]
2,71874,[Mukaradeeb],[Mukaradeeb]
3,70296,[Herbivore men],[Herbivore_men]
4,16578,[Shulin District],[Shulin_District]


In [41]:
pages = { 'asdf': 1}
for page in pages.items():
    print(page)

('asdf', 1)
